In [1]:
# There 2,000 images  dataset available on Kaggle, which contains 25,000 images.
# Here, we use a subset of the full dataset to decrease training time for Tutorial only!

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()


--2020-04-10 17:27:14--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.119.128, 2a00:1450:4013:c08::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.119.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   188MB/s    in 0.3s    

2020-04-10 17:27:15 (188 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our Testing cat pictures
Testing_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our Testing dog pictures
Testing_dogs_dir = os.path.join(validation_dir, 'dogs')

In [3]:
print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total Testing cat images:', len(os.listdir(Testing_cats_dir)))
print('total Testing dog images:', len(os.listdir(Testing_dogs_dir)))



total training cat images: 1000
total training dog images: 1000
total Testing cat images: 500
total Testing dog images: 500


In [0]:
from tensorflow.keras import layers
from tensorflow.keras import Model

img_input = layers.Input(shape=(150, 150, 3))
x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)


x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)


x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)


x = layers.Flatten()(x)

x = layers.Dense(512, activation='relu')(x)

output = layers.Dense(1, activation='sigmoid')(x)

# Create model:
# input = input feature map
# output = input feature map + stacked convolution/maxpooling layers + fully 
# connected layer + sigmoid output layer
model = Model(img_input, output)




In [5]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0     

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])


In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using val_datagen generator
validation_generator = val_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,  # 2000 images = batch_size * steps
      epochs=15,
      validation_data=validation_generator,
      validation_steps=50,  # 1000 images = batch_size * steps
      verbose=2)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/15
100/100 - 9s - loss: 0.7618 - acc: 0.5490 - val_loss: 0.6581 - val_acc: 0.6590
Epoch 2/15
100/100 - 8s - loss: 0.6152 - acc: 0.6760 - val_loss: 0.5800 - val_acc: 0.6830
Epoch 3/15
100/100 - 8s - loss: 0.5526 - acc: 0.7280 - val_loss: 0.5844 - val_acc: 0.7010
Epoch 4/15
100/100 - 8s - loss: 0.4771 - acc: 0.7620 - val_loss: 0.5495 - val_acc: 0.7210
Epoch 5/15
100/100 - 8s - loss: 0.4013 - acc: 0.8155 - val_loss: 0.5986 - val_acc: 0.7100
Epoch 6/15
100/100 - 8s - loss: 0.3213 - acc: 0.8585 - val_loss: 0.7351 - val_acc: 0.7250
Epoch 7/15
100/100 - 8s - loss: 0.2404 - acc: 0.9020 - val_loss: 0.8168 - val_acc: 0.7160
Epoch 8/15
100/100 - 8s - loss: 0.1915 - acc: 0.9235 - val_loss: 0.8110 - val_acc: 0.7170
Epoch 9/15
100/100 - 8s - loss: 0.1142 - acc: 0.9590 - val_loss: 0.9710 - val_acc: 0.7340
Epoch 10/15
100/100 - 8s - los